In [2]:
# ps53
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a3245682c7d11c975b63f850be770f0bfe5ac46523ce05f0113c8d1f52bde85b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


🌟 𝐏𝐘𝐒𝐏𝐀𝐑𝐊 𝐏𝐫𝐚𝐜𝐭𝐢𝐜𝐞/𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 📊
========================================
𝐀𝐯𝐞𝐫𝐚𝐠𝐞 𝐒𝐞𝐥𝐥𝐢𝐧𝐠 𝐏𝐫𝐢𝐜𝐞 𝐏𝐫𝐨𝐛𝐥𝐞𝐦

Write an pyspark code to find the average selling price for each product.
average_price should be rounded to 2 decimal places.


𝐒𝐜𝐡𝐞𝐦𝐚 𝐀𝐧𝐝 𝐃𝐚𝐭𝐚:
================
 # Prices table data
prices_data = [
 (1, '2019-02-17', '2019-02-28', 5),
 (1, '2019-03-01', '2019-03-22', 20),
 (2, '2019-02-01', '2019-02-20', 15),
 (2, '2019-02-21', '2019-03-31', 30)
]
# Define schema for Prices table
prices_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("start_date", StringType(), True),
 StructField("end_date", StringType(), True),
 StructField("price", IntegerType(), True)
])

 # UnitsSold table data
units_sold_data = [
 (1, '2019-02-25', 100),
 (1, '2019-03-01', 15),
 (2, '2019-02-10', 200),
 (2, '2019-03-22', 30)
]
# Define schema for UnitsSold table
units_sold_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("purchase_date", StringType(), True),
 StructField("units", IntegerType(), True)
])

In [3]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
# Initialize spark session
spark = SparkSession.builder.master('local').appName('PySpark-day4').getOrCreate()

In [5]:
spark

In [6]:
# define schema and data for price tbl
prices_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("start_date", StringType(), True),
 StructField("end_date", StringType(), True),
 StructField("price", IntegerType(), True)
])
 # Prices table data
prices_data = [
 (1, '2019-02-17', '2019-02-28', 5),
 (1, '2019-03-01', '2019-03-22', 20),
 (2, '2019-02-01', '2019-02-20', 15),
 (2, '2019-02-21', '2019-03-31', 30)
]

In [7]:
# define schema and data for units sold
units_sold_schema = StructType([
 StructField("product_id", IntegerType(), True),
 StructField("purchase_date", StringType(), True),
 StructField("units", IntegerType(), True)
])
# data
units_sold_data = [
 (1, '2019-02-25', 100),
 (1, '2019-03-01', 15),
 (2, '2019-02-10', 200),
 (2, '2019-03-22', 30)
]

In [8]:
# create df for prices
price_df = spark.createDataFrame(schema = prices_schema, data = prices_data)
price_df.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+



In [9]:
# create df for units sold
units_df = spark.createDataFrame(schema = units_sold_schema, data = units_sold_data)
units_df.show()

+----------+-------------+-----+
|product_id|purchase_date|units|
+----------+-------------+-----+
|         1|   2019-02-25|  100|
|         1|   2019-03-01|   15|
|         2|   2019-02-10|  200|
|         2|   2019-03-22|   30|
+----------+-------------+-----+



In [19]:
# join both df then filter out data based on purchase data between start and end date
join_df = price_df.join(units_df, on='product_id', how='inner')
result_df = join_df.filter(col('purchase_date').between(col('start_date'),col('end_date')))
result_df.show()

+----------+----------+----------+-----+-------------+-----+
|product_id|start_date|  end_date|price|purchase_date|units|
+----------+----------+----------+-----+-------------+-----+
|         1|2019-02-17|2019-02-28|    5|   2019-02-25|  100|
|         1|2019-03-01|2019-03-22|   20|   2019-03-01|   15|
|         2|2019-02-01|2019-02-20|   15|   2019-02-10|  200|
|         2|2019-02-21|2019-03-31|   30|   2019-03-22|   30|
+----------+----------+----------+-----+-------------+-----+



In [21]:
# create new col of price*units as total
result_df = result_df.withColumn('total', col('price')*col('units'))
result_df.show()

+----------+----------+----------+-----+-------------+-----+-----+
|product_id|start_date|  end_date|price|purchase_date|units|total|
+----------+----------+----------+-----+-------------+-----+-----+
|         1|2019-02-17|2019-02-28|    5|   2019-02-25|  100|  500|
|         1|2019-03-01|2019-03-22|   20|   2019-03-01|   15|  300|
|         2|2019-02-01|2019-02-20|   15|   2019-02-10|  200| 3000|
|         2|2019-02-21|2019-03-31|   30|   2019-03-22|   30|  900|
+----------+----------+----------+-----+-------------+-----+-----+



In [22]:
# group by product_id
result_df.groupBy('product_id').agg(round(sum('total')/sum('units'),2).alias('average_price')).show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+

